In [1]:
%load_ext tensorboard

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.init as nn_init
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.datasets import MNIST

In [3]:
train_data = MNIST("data/MNIST", download=True)
test_data = MNIST("data/MNIST", train=False, download=True)

In [4]:
train_data[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x7F3B280F0400>, 5)

In [5]:
train_data[0][0]

In [6]:
len(train_data)

60000

In [7]:
class MNISTDataset(Dataset):
    
    def __init__(self, dataset):
        self.data = dataset
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        pil_image, label = self.data[index]
        image_array = np.array(pil_image)
        image_flatten = image_array.flatten() / 255
        return image_flatten.astype(np.float32), label

In [8]:
train_dataset = MNISTDataset(train_data)
test_dataset = MNISTDataset(test_data)

In [9]:
class Model(nn.Module):
    
    def __init__(self, n_input, n_classes):
        super().__init__()
        self.fc1 = nn.Linear(n_input, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, n_classes)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [10]:
N_CLASSES = 10
N_INPUT = np.array(train_data[0][0]).size
BATCH_SIZE = 64
EPOCH = 20

In [11]:
model = Model(N_INPUT, N_CLASSES)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [13]:
torch.manual_seed(42)

for name, parameter in model.named_parameters():
    if "weight" in name:
        nn_init.xavier_normal_(parameter)

In [14]:
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

In [15]:
criteria = nn.CrossEntropyLoss()

In [16]:
model.cuda();

In [17]:
writer = SummaryWriter(log_dir="logs/mnist/")

In [18]:
def calc_accuracy(loader, model):
    ytrues = []
    ypreds = []
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            out = model(x) # B, C
            out_numpy = out.cpu().numpy()
            pred = np.argmax(out_numpy, axis=1)
            ytrues.extend(y.numpy())
            ypreds.extend(pred)
        metric = (np.array(ytrues) == np.array(ypreds)).mean()
    return metric

In [3]:
%tensorboard --logdir logs/mnist

In [21]:
for epoch in range(EPOCH):
    model.train()
    for i, (x, y) in enumerate(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = criteria(out, y)
        loss.backward()
        optimizer.step()
        if (i + 1) % 400 == 0:
            global_iteration = epoch * len(train_loader) + i
            writer.add_scalar("train/loss", loss.item(), global_step=global_iteration)
        scheduler.step()
    
    global_iteration = (epoch + 1) * len(train_loader)
    model.eval()
    test_metric = calc_accuracy(test_loader, model)
    train_metric = calc_accuracy(train_loader, model)
    writer.add_scalar("train/accuracy", train_metric, global_step=global_iteration)
    writer.add_scalar("test/accuracy", test_metric, global_step=global_iteration)